In [20]:
pd.DataFrame.agg

<function pandas.core.frame.DataFrame.aggregate(self, func=None, axis: 'Axis' = 0, *args, **kwargs)>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [2]:
ddf = dd.read_parquet("./data/parquet/data-*.parquet")

In [4]:
ddf.shape[0].compute()

23346088

In [7]:
only_year = lambda x: x[-4:]

In [13]:
ddf['Date'].apply(only_year, meta =('Date', 'object')).compute()

0        2013
1        2014
2        2014
3        2015
4        2013
         ... 
41177    2014
41178    2015
41179    2013
41180    2012
41181    2015
Name: Date, Length: 23346088, dtype: object

In [16]:
ddf.index.compute().unique()

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            46109, 46110, 46111, 46112, 46113, 46114, 46115, 46116, 46117,
            46118],
           dtype='int64', length=46119)

In [18]:
ddf.reset_index().drop("index", axis = 1).index.compute().unique()

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            46109, 46110, 46111, 46112, 46113, 46114, 46115, 46116, 46117,
            46118],
           dtype='int64', length=46119)